In [1]:
import pyprind
import pandas as pd 
import os

In [2]:
basepath = "../aclImdb"

In [3]:
labels = {'pos': 1, 'neg': 0}

In [25]:
# df = pd.DataFrame()
all_items = []
pbar = pyprind.ProgBar(50000)
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
                all_items.append({'review': txt, 'sentiment': labels[l]})
                # df.append([[txt, lables[l]]], ignore_index=True)
                pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:53


In [22]:
df = pd.DataFrame()
# all_items = []
pbar = pyprind.ProgBar(50000)
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r') as infile:
                txt = infile.read()
                # all_items.append({'review': txt, 'sentiment': labels[l]})
                df = df.append([[txt, labels[l]]], ignore_index=True)
                pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


KeyboardInterrupt: 

In [28]:
3+2

5

In [27]:
%%time
df = pd.DataFrame(all_items)

CPU times: user 71.7 ms, sys: 4.15 ms, total: 75.8 ms
Wall time: 94.7 ms


In [29]:
df.head()

,review,sentiment
0,"Based on an actual story, John Boorman shows t...",1
1,This is a gem. As a Film Four production - the...,1
2,"I really like this show. It has drama, romance...",1
3,This is the best 3-D experience Disney has at ...,1
4,"Of the Korean movies I've seen, only three had...",1


In [30]:
import numpy as np
np.random.seed(0)

df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False)

In [31]:
df = pd.read_csv('movie_data.csv')

In [32]:
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'
])

In [35]:
bag = count.fit_transform(docs)

In [36]:
bag

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [40]:
print(count.vocabulary_)

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [41]:
print(bag.toarray())

[[0 1 1 1 0 1 0]
 [0 1 0 0 1 1 1]
 [1 2 1 1 1 2 1]]


In [45]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)

In [53]:
tfidf.fit_transform(count.fit_transform(docs)).toarray()

array([[0.  , 0.43, 0.56, 0.56, 0.  , 0.43, 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.56, 0.43, 0.56],
       [0.4 , 0.48, 0.31, 0.31, 0.31, 0.48, 0.31]])

In [82]:
df.loc[6, 'review'][-150:]

"essive endings to a movie I have ever seen. For fans of Al Pacino and crime movies it's a must-see. For the rest of you it's highly recommended. 10/10"

In [83]:
for doc in df.review:
    if '<' in doc:
        break

In [88]:
doc[:170]

'My family and I normally do not watch local movies for the simple reason that they are poorly made, they lack the depth, and just not worth our time.<br /><br />The trail'

In [99]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticon_pattern= re.compile(r"(?::|;|=)(?:-)?(?:\)|\(|D|P)")
    emoticons = re.findall(emoticon_pattern, text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return text

In [100]:
preprocessor(doc)

'my family and i normally do not watch local movies for the simple reason that they are poorly made they lack the depth and just not worth our time the trailer of nasaan ka man caught my attention my daughter in law s and daughter s so we took time out to watch it this afternoon the movie exceeded our expectations the cinematography was very good the story beautiful and the acting awesome jericho rosales was really very good so s claudine barretto the fact that i despised diether ocampo proves he was effective at his role i have never been this touched moved and affected by a local movie before imagine a cynic like me dabbing my eyes at the end of the movie congratulations to star cinema way to go jericho and claudine '

In [101]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [102]:
df['review'] = df['review'].apply(preprocessor)

In [103]:
def tokenizer(text):
    return text.split()

In [107]:

runner_string = 'runners like running and thus they run'
tokenizer(runner_string)

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [106]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokeinzer_porter(text):
    return [porter.stem(word) for word in text.split()]


In [108]:
tokeinzer_porter(runner_string)

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [111]:
import nltk
from nltk.corpus import stopwords

In [124]:
stop = stopwords.words('english')

In [125]:
[w for w in tokeinzer_porter(runner_string) if w not in stop]

['runner', 'like', 'run', 'thu', 'run']

In [117]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [121]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [122]:
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

In [126]:
param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokeinzer_porter],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0],
    },
    {
        'vect__ngram_range': [(1,1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokeinzer_porter],
        'vect__use_idf': [False],
        'vect__norm': [None],
        'clf__penalty': ['l1', 'l2'],
        'clf__C': [1.0, 10.0, 100.0],
    }
]

In [143]:
param_grid = [
    {
        'vect__ngram_range': [(1, 1)],
        'vect__stop_words': [stop, None],
        'vect__tokenizer': [tokenizer, tokeinzer_porter],
        'clf__penalty': ['l1',],
        'clf__C': [1.0, ],
    },
]

In [148]:
%%time
lr_tfidf.fit(X_train, y_train)

CPU times: user 6.85 s, sys: 148 ms, total: 7 s
Wall time: 7.11 s


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling

In [144]:
lr_tfidf = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(random_state=1, solver='saga'))
])

In [145]:
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=2, verbose=1, n_jobs=-1)

In [149]:
%%time
gs_lr_tfidf.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
gs_lr_tfidf.best_params_

In [ ]:
gs_lr_tfidf.best_score_

In [ ]:
clf = gs_lr_tfidf.best_estimator_

In [ ]:
clf.score(X_test, y_test)

In [150]:
import re
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticon_pattern= re.compile(r"(?::|;|=)(?:-)?(?:\)|\(|D|P)")
    emoticons = re.findall(emoticon_pattern, text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))
    return [w for w in text.split() if w not in stop]

In [151]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [152]:
next(stream_docs(path='movie_data.csv'))

('"My family and I normally do not watch local movies for the simple reason that they are poorly made, they lack the depth, and just not worth our time.<br /><br />The trailer of ""Nasaan ka man"" caught my attention, my daughter in law\'s and daughter\'s so we took time out to watch it this afternoon. The movie exceeded our expectations. The cinematography was very good, the story beautiful and the acting awesome. Jericho Rosales was really very good, so\'s Claudine Barretto. The fact that I despised Diether Ocampo proves he was effective at his role. I have never been this touched, moved and affected by a local movie before. Imagine a cynic like me dabbing my eyes at the end of the movie? Congratulations to Star Cinema!! Way to go, Jericho and Claudine!!"',
 1)

In [154]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None 
    return docs, y

In [155]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)

In [159]:
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)

In [160]:
doc_stream = stream_docs(path = 'movie_data.csv')

In [161]:
import pyprind

In [162]:
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:29


In [163]:
X_test, y_test = get_minibatch(doc_stream, size=5000)

In [164]:
X_test = vect.transform(X_test)

In [165]:
clf.score(X_test, y_test)

0.8656

In [166]:
clf = clf.partial_fit(X_test, y_test)

In [167]:
count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)

In [168]:
X = count.fit_transform(df['review'].values)

In [169]:
from sklearn.decomposition import LatentDirichletAllocation

In [172]:
lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')

In [173]:
X_topics = lda.fit_transform(X)

In [174]:
lda.components_.shape

(10, 5000)

In [178]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx+1}")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))

Topic 1
comedy original horror black version
Topic 2
worst minutes script guy money
Topic 3
book dvd read version watched
Topic 4
family performance father beautiful mother
Topic 5
series episode tv kids episodes
Topic 6
murder police wife woman john
Topic 7
documentary camera audience human sense
Topic 8
music song songs musical dance
Topic 9
horror effects budget gore special
Topic 10
action war game fight hero


In [181]:
horror = X_topics[:,0].argsort()[::-1]

In [182]:
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(f"Horror movie #{iter_idx+1}")
    print(df['review'][movie_idx][:300])

Horror movie #1
house of dracula was made towards the end of universal s horror cycle of the 1940 s and i ve seen this a couple of times a mad doctor edelman is breeding plants for a serum that cures people count dracula arrives for a cure for his vampirism and lawrence talbot then comes to see if he can cure him f
Horror movie #2
despite all of the comparisons to house of frankenstein this movie outshines its predecessor and is one of the funnest monster movies in the universal monsters series and sadly the last o k so it doesn t exactly stack up to something like the bride of frankenstein as one a classic to revered by fans
Horror movie #3
 the man in the white suit is another feather in the cap of expert scottish director alexander mackendrick whisky galore sweet smell of success the star of the film is alec guinness whose comedies include kind hearts and coronets and the lavender hill mob guiness brings his usual class to the film a


In [183]:
import pickle
import os

In [184]:
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

In [185]:
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)

In [186]:
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)